## Preprocessing the Data

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, LabelEncoder
df = pd.read_csv('data/movies.csv')
df.head()
df.shape
df.columns
drop_cols =['movie_id','synopsis','New_York_Film_Critics_Circle_won_categories','Hollywood_Film_won_categories','Hollywood_Film_nominated_categories','Austin_Film_Critics_Association_won_categories','Austin_Film_Critics_Association_nominated_categories','Denver_Film_Critics_Society_won_categories','Denver_Film_Critics_Society_nominated_categories','Boston_Society_of_Film_Critics_won_categories','Boston_Society_of_Film_Critics_nominated_categories','New_York_Film_Critics_Circle_nominated_categories','Los_Angeles_Film_Critics_Association_won_categories','Los_Angeles_Film_Critics_Association_nominated_categories','Online_Film_Critics_Society_won_categories','Online_Film_Critics_Society_nominated_categories','People_Choice_won_categories','People_Choice_nominated_categories','London_Critics_Circle_Film_won_categories','London_Critics_Circle_Film_nominated_categories','American_Cinema_Editors_won_categories','American_Cinema_Editors_nominated_categories','Costume_Designers_Guild_won_categories','Costume_Designers_Guild_nominated_categories','Online_Film_Television_Association_won_categories','Online_Film_Television_Association_nominated_categories','Producers_Guild_won_categories','Producers_Guild_nominated_categories','Art_Directors_Guild_won_categories','Art_Directors_Guild_nominated_categories','Writers_Guild_won_categories','Writers_Guild_nominated_categories','Critics_Choice_won_categories','Critics_Choice_nominated_categories','Directors_Guild_won_categories','Directors_Guild_nominated_categories','Screen_Actors_Guild_won_categories','Screen_Actors_Guild_nominated_categories','BAFTA_won_categories','BAFTA_nominated_categories','Golden_Globes_won_categories','Golden_Globes_nominated_categories','Oscar_nominated_categories','genre','year','release_date','release_date.year', 'release_date.month', 'release_date.day-of-month', 'release_date.day-of-week']
df = df.drop(columns = drop_cols)

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
oe = OrdinalEncoder(categories = ['No','Yes'])
ss = StandardScaler()
le = LabelEncoder()

#Replacing columns with "Yes"/"No" values to 1/0
ord_cols = 'Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated'
ord_cols = [x for x in ord_cols.split(',')]
for col in ord_cols:
    df[col].replace({'Yes':1,'No':0}, inplace=True)
    
num_cols = 'duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_nominations,Oscar_nominated,Golden_Globes_nominated,BAFTA_won,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Hollywood_Film_won,Hollywood_Film_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated'
num_cols = [x for x in num_cols.split(',')]

ohe_data = pd.DataFrame(ohe.fit_transform(np.array(df['certificate'].replace({np.nan:"Missing"})).reshape(-1,1)))
ss_data = pd.DataFrame(ss.fit_transform(df[num_cols]))
le_data = pd.DataFrame(le.fit_transform(df['Oscar_Best_Picture_won']))

ss_data.columns=num_cols
le_data.columns=['Oscar_Best_Picture_won']
ohe_col_names = [('certificate'+x[2:]) for x in ohe.get_feature_names()]
ohe_data.columns=ohe_col_names

final_data = pd.concat([df['movie'],ohe_data,ss_data,df[ord_cols],le_data], axis=1)
final_data.to_csv('data/preprocessed_movies.csv', index=False)

### Reading the Data

In [14]:
df= pd.read_csv('data/preprocessed_movies.csv')
print(df.shape)
df.head()

(1235, 75)


,movie,certificate_G,certificate_Missing,certificate_Not Rated,certificate_PG,certificate_PG-13,certificate_R,certificate_TV-MA,certificate_TV-PG,certificate_Unrated,...,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_Best_Picture_won
0,Kate & Leopold,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,Chicken Run,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,Fantastic Four,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,Frida,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,The Lord of the Rings: The Fellowship of the Ring,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,1,0,0,0
